In [1]:
import numpy as np


In [2]:
def LU_Decompose(A):
    """
    Performs LU Decomposition on matrix A with partial pivoting.
    Args:
        A: The input matrix (n x n)
    Returns:
        L: The lower triangular matrix
        U: The upper triangular matrix
        P: The pivot matrix
    """
    n = A.shape[0]
    L = np.eye(n)
    U = A.copy()
    P = np.eye(n)

    for k in range(n):
        # Pivoting
        max_row = np.argmax(abs(U[k:, k])) + k
        if k != max_row:
            U[[k, max_row]] = U[[max_row, k]]
            P[[k, max_row]] = P[[max_row, k]]
            if k > 0:
                L[[k, max_row], :k] = L[[max_row, k], :k]

        # LU Decomposition
        for i in range(k + 1, n):
            L[i, k] = U[i, k] / U[k, k]
            U[i, k:] -= L[i, k] * U[k, k:]

    return L, U, P


In [10]:
def getLU(LU_of_M):
    """
    Compose the final LU matrices from the intermediate matrices.
    Args:
        LU_of_M: A tuple containing (L1_inv, S_hat, L3_inv, T, P1, P2)
    Returns:
        L_inv, U_inv, P: The final LU matrices
    """
    L1_inv, S_hat, L3_inv, U1_inv, T, P1M2, U3_inv, P1, P2 = LU_of_M

    # Construct the block matrix components for L_inv, U_inv, and P
    S = P2 @ S_hat
    # L_inv = np.block([[L1_inv, np.zeros_like(S_hat)],
    #                   [-np.dot(L3_inv, S_hat), L3_inv]])
    L_inv = np.block([[L1_inv, np.zeros_like(S_hat)],
                      -L3_inv @ S, L3_inv])

    # U_inv = np.block([[U1_inv, -np.dot(T, U3_inv)],
    #                   [np.zeros_like(T), U3_inv]])
    U_inv = np.block([[U1_inv, -T @ P1M2 @ U3_inv],
                      [np.zeros_like(T), U3_inv]])

    P = np.block([[P1, np.zeros_like(P2)],
                  [np.zeros_like(P1), P2]])

    return L_inv, U_inv, P


In [12]:
def BlockLU_Decompose(M):
    """
    Optimized Block-based LU Decomposition.
    Args:
        M: Input matrix to decompose
    Returns:
        LU_of_M: A tuple containing the intermediate LU decomposition matrices
    """
    # Assuming M is a 2x2 block matrix
    n = M.shape[0]
    mid = n // 2

    # Divide matrix M into sub-blocks
    M1 = M[:mid, :mid]
    M2 = M[:mid, mid:]
    M3 = M[mid:, :mid]
    M4 = M[mid:, mid:]

    # Base case: when M1 is small enough to perform standard LU
    M1Mid = M1.shape[0] // 2
    if M1Mid <= 1:
        L1, U1, P1 = LU_Decompose(M1)
        L1_inv = np.linalg.inv(L1)
        U1_inv = np.linalg.inv(U1)
        T = U1_inv @ L1_inv
        S_hat = M3 @ T
        M_hat = M4 - S_hat @ (P1 @ M2)
        L3, U3, P2 = LU_Decompose(M_hat)
        L3_inv = np.linalg.inv(L3)
        U3_inv = np.linalg.inv(U3)

    else:
        # Recursive decomposition
        LU_of_M1 = BlockLU_Decompose(M1)
        L1_inv, U1_inv, P1 = getLU(LU_of_M1)
        T = U1_inv @ L1_inv
        S_hat = M3 @ T
        M_hat = M4 - S_hat @ (P1 @ M2)
        LU_of_M_hat = BlockLU_Decompose(M_hat)
        L3_inv, U3_inv, P2 = getLU(LU_of_M_hat)
    return L1_inv, S_hat, L3_inv, U1_inv, T, P1@M2, U3_inv, P1, P2


In [13]:
def BlockInverse(A):
  n = A.shape[0]
  mid = n//2
  if mid <= 1:
    return np.linalg.inv(A)
  else:
    A1 = A[:mid, :mid]
    A2 = A[:mid, mid:]
    A3 = A[mid:, :mid]
    A4 = A[mid:, mid:]

    LU_of_A = BlockLU_Decompose(A)
    L1_inv, S_hat, L3_inv, U1_inv, T, P1A2, U3_inv, P1, P2 = LU_of_A
    X1 = T @ P1
    X2 = U3_inv @ L3_inv
    Y1 = X1 @ A2
    Y2 = X2 @ A3
    A_inv_4 = X2
    A_inv_3 = -Y2 @ X1
    A_inv_2 = -Y1 @ X2
    A_inv_1 = X1 - Y1 @ A_inv_3
    return np.block([[A_inv_1, A_inv_2], [A_inv_3, A_inv_4]])
    return A_inv

In [19]:
A = np.array([[1,2,3],[4,5,6],[7,2,9]], dtype=float)
print(A)
print(BlockInverse(A))

[[1. 2. 3.]
 [4. 5. 6.]
 [7. 2. 9.]]
[[-0.91666667  0.33333333  0.08333333]
 [-0.16666667  0.33333333 -0.16666667]
 [ 0.75       -0.33333333  0.08333333]]
